In [51]:
import plotly_express as px
import pandas as pd
import requests
import mysql.connector
import math
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="emobilityapp2")
from datetime import datetime

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="ladestationen"
)
mycursor = mydb.cursor()

In [3]:
users=requests.get("https://api.plugsurfing.com/mfund/users")

print(users.text)

{"lowUsage":{"sessionsAverage":0,"stationsAverage":0},"highUsage":{"sessionsAverage":0,"stationsAverage":0}}


In [4]:
def insert_stations(val):
    try:
        mySql_insert_query = "INSERT INTO stations (id, latitude, longitude, utilization, provider) VALUES "+str(val)
        mycursor.execute(mySql_insert_query)
        mydb.commit()
        print(mycursor.rowcount, "station inserted.")
    except mysql.connector.Error as error:
        print("Failed to insert into MySQL table {}".format(error))

In [5]:
def insert_connectors(conns,stationid):
    for con in conns:
        try:
            id=con['id']
            speed=con['speed']
            powerType=con['powerType']
            val=(id,speed,powerType,stationid)
            mySql_insert_query = "INSERT INTO connectors (id, speed, powerType, stationid) VALUES "+str(val)
            mycursor.execute(mySql_insert_query)
            mydb.commit()
            print(mycursor.rowcount, "connector inserted.")
        except mysql.connector.Error as error:
            print("Failed to insert into MySQL table {}".format(error))

In [ ]:
def load_all():
    r=requests.get("https://api.plugsurfing.com/mfund/stations?page=1&limit=1")
    pages=math.ceil(int(r.headers['X-Total-Count'])/100)
    for page in range(1,pages+1):
        r=requests.get("https://api.plugsurfing.com/mfund/stations?page="+str(page)+"&limit=100")
        for i in range(len(r.json())):
            print("page: "+ str(page)+ " i= "+str(i))
            id=r.json()[i]['id']
            latitude=r.json()[i]['latitude']
            longitude=r.json()[i]['longitude']
            utilization=r.json()[i]['utilization']
            insert_address(latitude,longitude)
            provider=''
            val=(id, latitude, longitude, utilization, provider)
            insert_stations(val)
            conns=r.json()[i]['connectors']
            print("Länge von cons :" + str(len(conns)))
            insert_connectors(conns,id)

In [6]:
def get_ad_data(ad,str):
    if ad.get(str)==None:
        return None
    else:
        return ad.get(str)

In [7]:
def insert_address(latitude,longitude):
    try:
        location = geolocator.reverse(str(latitude)+","+str(longitude))
        if location==None:
            return
        ad=location.raw['address']
        street=get_ad_data(ad,'road')
        if street != None:
            street=street.replace("'", "''")
            street=street[:30]
        house_number=get_ad_data(ad,'house_number')
        postcode=get_ad_data(ad,'postcode')
        state=get_ad_data(ad,'state')
        if location.raw['address'].get('city')!=None:
            city=ad.get('city')
        elif location.raw['address'].get('town')!=None:
            city=ad.get('town')
        elif location.raw['address'].get('village')!=None:
            city=ad.get('village')
        else:
            city=get_ad_data(ad,'county')
        val=(latitude,longitude,street,house_number,postcode,city,state)
        print(val)
        mySql_insert_query = "INSERT INTO address (latitude, longitude, street, house_number, postcode, city, state) VALUES ({},{},{},{},{},{},{})".format(val[0],val[1],"'"+str(val[2])+"'" if val[2] else "NULL","'"+str(val[3])+"'" if val[3] else "NULL","'"+str(val[4])+"'" if val[4] else "NULL","'"+str(val[5])+"'" if val[5] else "NULL","'"+str(val[6])+"'" if val[6] else "NULL")
        mycursor.execute(mySql_insert_query)
        mydb.commit()
        print(mycursor.rowcount, " address inserted.")
    except mysql.connector.Error as error:
        print("Failed to insert into MySQL table {}".format(error))

In [8]:
def sel_place(kind,place):
    mycursor.execute("SELECT s.id, s.latitude,s.longitude FROM stations as s, address as a where s.latitude=a.latitude and s.longitude=a.longitude and {}='{}'".format(kind,place))
    myresult = mycursor.fetchall()
    df=pd.DataFrame(data=myresult,columns=['id','lat','lon'])
    print("There are "+ str(len(myresult)) + " stations in "+ place +".")
    fig=px.scatter_mapbox(df, lat='lat', lon='lon', hover_name='id', zoom=10)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()
    

In [22]:
def sel_place_w_conn(kind,place):
    mycursor.execute("SELECT s.id, s.latitude,s.longitude, count(c.id) FROM stations as s, address as a, connectors as c where c.stationid=s.id and s.latitude=a.latitude and s.longitude=a.longitude and {}='{}' group by s.id ".format(kind,place))
    myresult = mycursor.fetchall()
    df=pd.DataFrame(data=myresult,columns=['id','lat','lon','number_of_connectors'])
    print("There are "+ str(len(myresult)) + " stations in "+ place +".")
    fig=px.scatter_mapbox(df, lat='lat', lon='lon', size='number_of_connectors', zoom=10)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()
    

In [ ]:
sel_place_w_conn('city','Berlin')

In [94]:
def run():
    print('Press 1 to show total amount.\nPress 2 to show stations at location.\nPress 3 to show stations at location with number of connectors.\nPress 4 to show usage.\nPress 5 to check for new staions.')
    inp=int(input())
    if inp==1:
        r=requests.get("https://api.plugsurfing.com/mfund/stations?page=1&limit=1")
        print(r.headers['X-Total-Count'])
    if inp==2:
        print('what kind of location?')
        kind=input()
        print('what location?')
        place=input()
        sel_place(kind,place)
    if inp==3:
        print('what kind of location?')
        kind=input()
        print('what location?')
        place=input()
        sel_place_w_conn(kind,place)
    if inp==4:
        r=requests.get("https://api.plugsurfing.com/mfund/users")
        now = datetime.now()
        date = now.strftime('%Y-%m-%d %H:%M:%S')
        val=(date,r.json()['lowUsage']['sessionsAverage'],r.json()['lowUsage']['stationsAverage'])
        print('Average session duration: '+ str(r.json()['lowUsage']['sessionsAverage']))
        print('Average stations usage: '+ str(r.json()['lowUsage']['stationsAverage']))
        mycursor.execute("INSERT INTO users (date,sessions,stations) VALUES ('{}',{},{})".format(date,val[1],val[2]))
        mydb.commit()
        print('----------')
        print('Recorded data:')
        mycursor.execute("SELECT * FROM users")
        myresult = mycursor.fetchall()
        for d in myresult:
            print(d)
    if inp==5:
        r=requests.get("https://api.plugsurfing.com/mfund/stations?page=1&limit=1")
        n=r.headers['X-Total-Count']
        mycursor.execute("SELECT * FROM stations")
        myresult = mycursor.fetchall()
        
        if len(myresult)==int(n):
            print('no new data')
        else:
            load_all()

In [108]:
run()

Press 1 to show total amount.
Press 2 to show stations at location.
Press 3 to show stations at location with number of connectors.
Press 4 to show usage.
Press 5 to check for new staions.
5
no new data
